<a href="https://colab.research.google.com/github/Hattapoglu-Ebru/Listt/blob/main/e_to_do_list.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
To-Do List Console App
Ebru Hattapoglu

- Main menu: List / Add / Edit / Delete / Exit
- Prevent empty tasks; numbered listing; invalid indices handled
- Error handling with try/except (I/O, type conversions, date parse)
- Persistent storage in a UTF-8 text file (JSON), saved after every change
- Modular functions (add/delete/edit/list + file load/save)
- Simple console UI
- completed status toggle
- priority (1=High, 2=Medium, 3=Low)
- due_date (YYYY-MM-DD)
- created_at auto-set to today's date and stays fixed
- sorting options
- import/export (JSON/CSV)
- Google Colab helper: upload files from local machine (if running in Colab)
"""

import json
import os
import csv
from datetime import datetime

try:
    from google.colab import files
    IN_COLAB = True
except Exception:
    IN_COLAB = False

DATA_FILE = "tasks.json"

def today_str():
    return datetime.now().strftime("%Y-%m-%d")

def clear_console():
    print("\n" * 2)

def press_enter():
    input("Press Enter to continue...")

def safe_int(prompt, low=None, high=None):
    """Get an integer safely; enforce optional [low, high] bounds."""
    while True:
        s = input(prompt).strip()
        try:
            n = int(s)
            if low is not None and n < low:
                print(f"Please enter a number >= {low}.")
                continue
            if high is not None and n > high:
                print(f"Please enter a number <= {high}.")
                continue
            return n
        except ValueError:
            print("Please enter a valid number.")

def non_empty_text(prompt, default=None):
    """Get non-empty text; if default provided and user leaves blank, use default."""
    while True:
        s = input(prompt).strip()
        if s:
            return s
        if default is not None:
            return default
        print("Empty text is not allowed.")

def date_or_blank(prompt, default=None):
    """Get YYYY-MM-DD or blank (returns default)."""
    while True:
        s = input(prompt).strip()
        if not s:
            return default
        try:
            datetime.strptime(s, "%Y-%m-%d")
            return s
        except ValueError:
            print("Use date format YYYY-MM-DD (e.g., 2025-09-16), or leave blank.")

def load_tasks(path=DATA_FILE):
    """Load tasks from UTF-8 JSON file; return [] on first run or parse errors."""
    if not os.path.exists(path):
        print("Task file not found. A new list will be created.")
        return []
    try:
        with open(path, "r", encoding="utf-8") as f:
            data = json.load(f)
            if isinstance(data, list):
                return data
            print("Unexpected file format. Starting with an empty list.")
            return []
    except (json.JSONDecodeError, OSError) as e:
        print(f"Read error: {e}. Starting with an empty list.")
        return []

def save_tasks(tasks, path=DATA_FILE):
    """Persist tasks to UTF-8 JSON file after every change (spec requirement)."""
    try:
        with open(path, "w", encoding="utf-8") as f:
            json.dump(tasks, f, ensure_ascii=False, indent=2)
        print("Tasks saved.")
    except OSError as e:
        print(f"Write error: {e}. Check permissions/disk space.")

def make_task(text, priority=2, due_date=None, completed=False, created_at=None):
    """
    priority: 1=High, 2=Medium, 3=Low
    due_date: 'YYYY-MM-DD' or None
    created_at: set automatically to today if not provided; remains constant
    """
    return {
        "text": text,
        "completed": bool(completed),
        "priority": int(priority),
        "due_date": due_date,
        "created_at": created_at or today_str()
    }

def normalize_task(t):
    """Normalize incoming tasks (e.g., when importing) and fix missing/invalid fields."""
    text = str(t.get("text", "")).strip()
    if not text:
        return None
    completed = bool(t.get("completed", False))
    try:
        priority = int(t.get("priority", 2))
    except Exception:
        priority = 2
    due_date = t.get("due_date") or None
    created_at = t.get("created_at") or today_str()

    def _ok_date(s):
        if not s:
            return None
        try:
            datetime.strptime(s, "%Y-%m-%d")
            return s
        except Exception:
            return None

    due_date = _ok_date(due_date)
    created_at = _ok_date(created_at) or today_str()

    return make_task(text, priority, due_date, completed, created_at)

def priority_label(p):
    return {1: "High", 2: "Medium", 3: "Low"}.get(p, "Medium")

def print_tasks(tasks):
    print("\n--- TASK LIST ---")
    if not tasks:
        print("No tasks yet.")
        print("-----------------\n")
        return
    for i, t in enumerate(tasks, start=1):
        status = "✔" if t.get("completed") else "✗"
        label = priority_label(t.get("priority", 2))
        due = t.get("due_date") or "-"
        created = t.get("created_at") or "-"
        print(f"{i}. [{status}] ({label}) {t.get('text')}  | Due: {due} | Created: {created}")
    print("-----------------\n")

def add_task(tasks):
    text = non_empty_text("New task: ")
    prio = 2
    if input("Set priority? (y/N): ").strip().lower() == "y":
        prio = safe_int("Priority (1=High, 2=Medium, 3=Low): ", low=1, high=3)
    due = None
    if input("Set due date? (y/N): ").strip().lower() == "y":
        due = date_or_blank("Due date (YYYY-MM-DD) (blank = none): ", default=None)

    tasks.append(make_task(text, prio, due))
    print(f"Added: '{text}' (created_at: {tasks[-1]['created_at']}).")

def delete_task(tasks):
    if not tasks:
        print("List is empty, nothing to delete.")
        return
    print_tasks(tasks)
    idx = safe_int("Task number to delete: ", low=1, high=len(tasks))
    removed = tasks.pop(idx - 1)
    print(f"Deleted: '{removed.get('text')}'.")

def edit_task(tasks):
    if not tasks:
        print("List is empty, nothing to edit.")
        return
    print_tasks(tasks)
    idx = safe_int("Task number to edit: ", low=1, high=len(tasks))
    t = tasks[idx - 1]
    old = t.get("text", "")
    new_text = non_empty_text(f"New text ({old}): ", default=old)
    t["text"] = new_text
    print("Task updated.")

def toggle_completed(tasks):
    if not tasks:
        print("List is empty.")
        return
    print_tasks(tasks)
    idx = safe_int("Task number to toggle completed: ", low=1, high=len(tasks))
    t = tasks[idx - 1]
    t["completed"] = not t.get("completed", False)
    print(f"Status: {'Completed' if t['completed'] else 'In progress'}")

def change_priority(tasks):
    if not tasks:
        print("List is empty.")
        return
    print_tasks(tasks)
    idx = safe_int("Task number to change priority: ", low=1, high=len(tasks))
    new_p = safe_int("New priority (1=High, 2=Medium, 3=Low): ", low=1, high=3)
    tasks[idx - 1]["priority"] = new_p
    print("Priority updated.")

def change_due_date(tasks):
    if not tasks:
        print("List is empty.")
        return
    print_tasks(tasks)
    idx = safe_int("Task number to change due date: ", low=1, high=len(tasks))
    new_due = date_or_blank("New due date (YYYY-MM-DD) (blank=clear): ", default=None)
    tasks[idx - 1]["due_date"] = new_due
    print("Due date updated.")

def sort_tasks(tasks):
    if not tasks:
        print("List is empty, nothing to sort.")
        return
    print("""
Sorting options:
1) Incomplete first, then completed
2) By priority (High -> Medium -> Low)
3) By due date (earlier first), blanks last
4) Combined: (Incomplete -> Priority -> Due date)
""")
    choice = safe_int("Your choice (1-4): ", low=1, high=4)

    def due_key(t):
        val = t.get("due_date")
        if not val:
            return datetime.max
        try:
            return datetime.strptime(val, "%Y-%m-%d")
        except Exception:
            return datetime.max

    if choice == 1:
        tasks.sort(key=lambda x: x.get("completed", False))  # False(0) first, True(1) later
    elif choice == 2:
        tasks.sort(key=lambda x: x.get("priority", 2))       # 1 first
    elif choice == 3:
        tasks.sort(key=due_key)
    else:
        tasks.sort(key=lambda x: (x.get("completed", False), x.get("priority", 2), due_key(x)))
    print("Sorted.")

def colab_upload():
    if not IN_COLAB:
        print("This option is available only inside Google Colab.")
        return
    print("Choose JSON/CSV from your computer. It will be uploaded to the working directory.")
    files.upload()
    print("Uploaded. Now use '7) Import (JSON/CSV)' and provide the file name to ingest.")

def import_tasks(tasks):
    print("Enter a file path or name to import (e.g., 'external.json' or 'external.csv').")
    path = input("Path/name: ").strip()
    if not path:
        print("No file name provided.")
        return
    if not os.path.exists(path):
        print("File not found. Ensure it is in the working directory.")
        return

    ext = os.path.splitext(path)[1].lower()
    incoming = []
    try:
        if ext == ".json":
            with open(path, "r", encoding="utf-8") as f:
                data = json.load(f)
            if not isinstance(data, list):
                print("JSON must be a list of task objects.")
                return
            for raw in data:
                nt = normalize_task(raw)
                if nt:
                    incoming.append(nt)

        elif ext == ".csv":
            with open(path, "r", encoding="utf-8") as f:
                reader = csv.DictReader(f)
                for row in reader:
                    c_raw = row.get("completed", "").strip().lower()
                    completed = True if c_raw in ("1", "true", "yes", "y") else False
                    priority = row.get("priority", "2") or "2"
                    raw = {
                        "text": row.get("text", ""),
                        "completed": completed,
                        "priority": priority,
                        "due_date": row.get("due_date") or None,
                        "created_at": row.get("created_at") or None
                    }
                    nt = normalize_task(raw)
                    if nt:
                        incoming.append(nt)
        else:
            print("Unsupported extension. Use .json or .csv.")
            return
    except Exception as e:
        print(f"Import error: {e}")
        return

    if not incoming:
        print("No valid tasks found in file.")
        return

    print(f"Read {len(incoming)} task(s).")
    print("Merge method:")
    print("1) Append to existing list")
    print("2) REPLACE existing list with imported data")
    choice = safe_int("Your choice (1-2): ", low=1, high=2)

    if choice == 1:
        tasks += incoming
        print("Imported tasks appended.")
    else:
        tasks.clear()
        tasks.extend(incoming)
        print("Existing list replaced by imported tasks.")

def export_tasks(tasks):
    if not tasks:
        print("List is empty, nothing to export.")
        return
    print("Export format: 1) JSON  2) CSV")
    choice = safe_int("Your choice (1-2): ", low=1, high=2)
    filename = input("Output file name (e.g., out.json / out.csv): ").strip()
    if not filename:
        print("No file name provided.")
        return
    ext = os.path.splitext(filename)[1].lower()

    try:
        if choice == 1:
            if ext != ".json":
                filename += ".json"
            with open(filename, "w", encoding="utf-8") as f:
                json.dump(tasks, f, ensure_ascii=False, indent=2)
            print(f"Exported to JSON: '{filename}'")
        else:
            if ext != ".csv":
                filename += ".csv"
            fields = ["text", "completed", "priority", "due_date", "created_at"]
            with open(filename, "w", encoding="utf-8", newline="") as f:
                writer = csv.DictWriter(f, fieldnames=fields)
                writer.writeheader()
                for t in tasks:
                    writer.writerow({
                        "text": t.get("text", ""),
                        "completed": "1" if t.get("completed") else "0",
                        "priority": t.get("priority", 2),
                        "due_date": t.get("due_date") or "",
                        "created_at": t.get("created_at") or ""
                    })
            print(f"Exported to CSV: '{filename}'")
    except OSError as e:
        print(f"Write error: {e}")

# -- Menu ---
def advanced_menu(tasks):
    while True:
        print("""
--- ADVANCED MENU ---
1. Toggle Completed
2. Change Priority
3. Change Due Date
4. Sort Tasks
5. Back
""")
        choice = safe_int("Your choice (1-5): ", low=1, high=5)
        clear_console()

        if choice == 1:
            toggle_completed(tasks); save_tasks(tasks); press_enter()
        elif choice == 2:
            change_priority(tasks); save_tasks(tasks); press_enter()
        elif choice == 3:
            change_due_date(tasks); save_tasks(tasks); press_enter()
        elif choice == 4:
            sort_tasks(tasks); save_tasks(tasks); press_enter()
        else:
            return

def main_menu():
    clear_console()
    print("Welcome to the To-Do List App!")

    tasks = load_tasks()
    if tasks:
        print("Tasks loaded successfully.")
    else:
        print("Starting with a new task list.")

    while True:
        print(f"""
--- TO-DO LIST APP ---
1. List Tasks
2. Add Task
3. Edit Task
4. Delete Task
5. Advanced (completed/priority/due/sort)
6. Colab: Upload File from Computer{' (N/A here)' if not IN_COLAB else ''}
7. Import (JSON/CSV)
8. Export (JSON/CSV)
9. Exit
""")
        choice = safe_int("Your choice (1-9): ", low=1, high=9)
        clear_console()

        if choice == 1:
            print_tasks(tasks); press_enter()
        elif choice == 2:
            add_task(tasks); save_tasks(tasks); press_enter()
        elif choice == 3:
            edit_task(tasks); save_tasks(tasks); press_enter()
        elif choice == 4:
            delete_task(tasks); save_tasks(tasks); press_enter()
        elif choice == 5:
            advanced_menu(tasks)
        elif choice == 6:
            colab_upload(); press_enter()
        elif choice == 7:
            import_tasks(tasks); save_tasks(tasks); press_enter()
        elif choice == 8:
            export_tasks(tasks); press_enter()
        elif choice == 9:
            print("Exiting...")
            save_tasks(tasks)
            break

if __name__ == "__main__":
    try:
        main_menu()
    except KeyboardInterrupt:
        print("\nInterrupted. Bye!")





Welcome to the To-Do List App!
Tasks loaded successfully.

--- TO-DO LIST APP ---
1. List Tasks
2. Add Task
3. Edit Task
4. Delete Task
5. Advanced (completed/priority/due/sort)
6. Colab: Upload File from Computer
7. Import (JSON/CSV)
8. Export (JSON/CSV)
9. Exit





--- TASK LIST ---
1. [✗] (High) go to school  | Due: - | Created: 2025-09-16
-----------------

Task updated.
Tasks saved.
